# Study notes about Multi-Model Recommender

Assume we have 5 users and 6 products.

Matrix A will be the purchase history matrix

In [152]:
import numpy as np

## Co-occurrence matrix

A'A is the co-occurence matrix. It is an items x items matrix.  Each value represents the number of times two items co-occur (ignore the diagnoal values)

Examples:
- item 1 and item 3 co-occur twice (in user1 and user2)
- item 2 and item 4 co-occur twice (in user3 and user5)

In [153]:
A = np.matrix([[1,0,1,0,0,0], [1,0,1,0,1,0], [0,1,1,1,0,0], [0,1,0,1,0,1], [0,0,0,0,0,0]]) # History of purchase
print "Purchase History:\n", A, "\n"

AtA = A.transpose() * A
np.fill_diagonal(AtA, 0) # Fill the diagnonal with zeros, as they should be ignored.
print "Co-occurrence Matrix:\n", AtA, "\n"

Purchase History:
[[1 0 1 0 0 0]
 [1 0 1 0 1 0]
 [0 1 1 1 0 0]
 [0 1 0 1 0 1]
 [0 0 0 0 0 0]] 

Co-occurrence Matrix:
[[0 0 2 0 1 0]
 [0 0 1 2 0 1]
 [2 1 0 1 1 0]
 [0 2 1 0 0 1]
 [1 0 1 0 0 0]
 [0 1 0 1 0 0]] 



**Simple user-centric recommendation**

( **Note**: all LLR analysis are skipped in order to keep things simple here. )

If a user is viewing item 1, what should we recommend?

In [154]:
h = np.matrix([1,0,0,0,0,0])
r = h * AtA  # Note: AtA * h' will produce the same result
print r

[[0 0 2 0 1 0]]


The above result tells us that we should recommend item 3, and then item 5.  It's based on the co-occurrence matrix:
- Two people who purchased item 1 also purchased item 3.
- One person who purchased item 1 also purchased item 5

This is similar to the "people who purchased this also purchase" list.

Now let's say a user comes in, and in her history she has purchased item 1 and item 2 in the past.  What should we recommend?

In [155]:
print "Purchase History:\n", A, "\n"
p = np.matrix([1,1,0,0,0,0])
print "History: item 1 and item 2; recommendation:\n", p * AtA 

Purchase History:
[[1 0 1 0 0 0]
 [1 0 1 0 1 0]
 [0 1 1 1 0 0]
 [0 1 0 1 0 1]
 [0 0 0 0 0 0]] 

History: item 1 and item 2; recommendation:
[[0 0 3 2 1 1]]


Based on the result, we should recommend item 3, 4 and 5 (in that order).

Try one more time with user's history = item 1 and item 5.

In [156]:
print "Purchase History:\n", A, "\n"
p = np.matrix([1,0,0,0,1,0])
print "History: item 1 and item 5; recommendation:\n", p * AtA 

Purchase History:
[[1 0 1 0 0 0]
 [1 0 1 0 1 0]
 [0 1 1 1 0 0]
 [0 1 0 1 0 1]
 [0 0 0 0 0 0]] 

History: item 1 and item 5; recommendation:
[[1 0 3 0 1 0]]


###Sidetrack: Vector multiplication and kNN using [cosine distance][1]
[1]: http://docs.scipy.org/doc/scipy-0.16.0/reference/generated/scipy.spatial.distance.cosine.html "Cosine Distance"

In [157]:
print "Co-occurrence Matrix:\n", AtA, "\n"
print "A user's purchase history:", p, "\n"

from scipy.spatial import distance
for r in range(6):
    dist = distance.cosine(p, AtA[r])
    print "Item", r + 1, "- cosine distance with p =", dist

Co-occurrence Matrix:
[[0 0 2 0 1 0]
 [0 0 1 2 0 1]
 [2 1 0 1 1 0]
 [0 2 1 0 0 1]
 [1 0 1 0 0 0]
 [0 1 0 1 0 0]] 

A user's purchase history: [[1 0 0 0 1 0]] 

Item 1 - cosine distance with p = 0.683772233983
Item 2 - cosine distance with p = 1.0
Item 3 - cosine distance with p = 0.198216274263
Item 4 - cosine distance with p = 1.0
Item 5 - cosine distance with p = 0.5
Item 6 - cosine distance with p = 1.0


Based on nearest neighbour, the closest items are 3, 5, and 1.  That matches our earlier results.

##Cross Co-occurrence##

In addition to purchase history, we also have Bookmark history of the 6 products.

*Recommend a purchase based on bookmark history*

In [158]:
print "Purchase History:\n", A, "\n"

B = np.matrix([[0,0,0,1,0,0], [0,0,0,1,0,0], [0,1,0,0,0,0], [0,1,0,0,0,0], [1,0,0,0,1,1]]) # History of bookmark
print "Bookmark History:\n", B, "\n"
BtA = B.transpose() * A

print "Bookmark: 4  -", np.matrix([0,0,0,1,0,0]) * BtA
print "Bookmark: 2  -",  np.matrix([0,1,0,0,0,0]) * BtA
print "Bookmark: 1  -",  np.matrix([1,0,0,0,0,0]) * BtA
print "Bookmark: 6  -",  np.matrix([0,0,0,0,0,1]) * BtA
print "Bookmark: 1&2-",  np.matrix([1,1,0,0,0,0]) * BtA

Purchase History:
[[1 0 1 0 0 0]
 [1 0 1 0 1 0]
 [0 1 1 1 0 0]
 [0 1 0 1 0 1]
 [0 0 0 0 0 0]] 

Bookmark History:
[[0 0 0 1 0 0]
 [0 0 0 1 0 0]
 [0 1 0 0 0 0]
 [0 1 0 0 0 0]
 [1 0 0 0 1 1]] 

Bookmark: 4  - [[2 0 2 0 1 0]]
Bookmark: 2  - [[0 2 1 2 0 1]]
Bookmark: 1  - [[0 0 0 0 0 0]]
Bookmark: 6  - [[0 0 0 0 0 0]]
Bookmark: 1&2- [[0 2 1 2 0 1]]


Please note that in the last three cases, only user 5 have bookmarked items 1 and 5. But since he hasn't purchased any item, so we **cannot** correlate those two bookmarks to any purchase, and thus they have **no** effect on the recommendation. 

###Recommend a purchase based on both purchase history and bookmark history

In [159]:
print "Purchase History:\n", A, "\n"
print "Bookmark History:\n", B, "\n"

p = np.matrix([1,0,1,0,0,0])
b = np.matrix([0,1,0,0,0,0])
r = p*AtA + b*BtA

print "A user's purchase:", p
print "Recommendations:  ", p*AtA, "\n"
print "A user's bookmark:", b
print "Recommendations:  ", b*BtA, "\n"
print "Combined:         ", p*AtA + b*BtA, "\n"


Purchase History:
[[1 0 1 0 0 0]
 [1 0 1 0 1 0]
 [0 1 1 1 0 0]
 [0 1 0 1 0 1]
 [0 0 0 0 0 0]] 

Bookmark History:
[[0 0 0 1 0 0]
 [0 0 0 1 0 0]
 [0 1 0 0 0 0]
 [0 1 0 0 0 0]
 [1 0 0 0 1 1]] 

A user's purchase: [[1 0 1 0 0 0]]
Recommendations:   [[2 1 2 1 2 0]] 

A user's bookmark: [[0 1 0 0 0 0]]
Recommendations:   [[0 2 1 2 0 1]] 

Combined:          [[2 3 3 3 2 1]] 



###What if we have users' gender information?

Let's say we have a new matrix, G, which record the gender information.  The columns represent Male and Female.

In [160]:
print "Purchase History:\n", A, "\n"
G = np.matrix([[1,0], [0,1], [0,1], [1,0], [0,1]])
print "Gender info:\n", G, "\n"

Purchase History:
[[1 0 1 0 0 0]
 [1 0 1 0 1 0]
 [0 1 1 1 0 0]
 [0 1 0 1 0 1]
 [0 0 0 0 0 0]] 

Gender info:
[[1 0]
 [0 1]
 [0 1]
 [1 0]
 [0 1]] 



*For a female customer, what should we recommend?*

In [161]:
GtA = G.transpose() * A
g = np.matrix([0,1])
print "For a female, we recommend: ", g * GtA

For a female, we recommend:  [[1 1 2 1 1 0]]


### Recommend based on purchase history, bookmark history and gender info

In [162]:
print "Purchase History:\n", A, "\n"
print "Bookmark History:\n", B, "\n"
print "Gender info:\n", G, "\n"
print "A user's purchase:", p
print "Recommendations:  ", p*AtA, "\n"
print "A user's bookmark:", b
print "Recommendations:  ", b*BtA, "\n"
print "A user's gender:  ", g
print "Recommendations:  ", g*GtA, "\n"

print "Combined (two):   ", p*AtA + b*BtA, "\n"
print "Combined (all):   ", p*AtA + b*BtA + g*GtA, "\n"


Purchase History:
[[1 0 1 0 0 0]
 [1 0 1 0 1 0]
 [0 1 1 1 0 0]
 [0 1 0 1 0 1]
 [0 0 0 0 0 0]] 

Bookmark History:
[[0 0 0 1 0 0]
 [0 0 0 1 0 0]
 [0 1 0 0 0 0]
 [0 1 0 0 0 0]
 [1 0 0 0 1 1]] 

Gender info:
[[1 0]
 [0 1]
 [0 1]
 [1 0]
 [0 1]] 

A user's purchase: [[1 0 1 0 0 0]]
Recommendations:   [[2 1 2 1 2 0]] 

A user's bookmark: [[0 1 0 0 0 0]]
Recommendations:   [[0 2 1 2 0 1]] 

A user's gender:   [[0 1]]
Recommendations:   [[1 1 2 1 1 0]] 

Combined (two):    [[2 3 3 3 2 1]] 

Combined (all):    [[3 4 5 4 3 1]] 



##Similar Items



In [163]:
print "Purchase History:\n", A, "\n"
print "Purchase co-occurrence:\n", AtA, "\n"

Purchase History:
[[1 0 1 0 0 0]
 [1 0 1 0 1 0]
 [0 1 1 1 0 0]
 [0 1 0 1 0 1]
 [0 0 0 0 0 0]] 

Purchase co-occurrence:
[[0 0 2 0 1 0]
 [0 0 1 2 0 1]
 [2 1 0 1 1 0]
 [0 2 1 0 0 1]
 [1 0 1 0 0 0]
 [0 1 0 1 0 0]] 



Take item 4 as an example.  

Its row in AtA is `[0 2 1 0 0 1]`.  So it occurs the most with item 2, and also with 3 and 6.  So items 2, 3 and 6 are similar to it (based on purchase history).  

We can obtain the same result by multiplying AtA with an item vector.

In [164]:
i = np.matrix([0,0,0,1,0,0])
i * AtA

matrix([[0, 2, 1, 0, 0, 1]])

So i * AtA produces the "similar item" vector based on just the purchase history.

What if we use both purchase history and bookmark history.  Please note that we cannot use gender, because it's a user's attribute, and we only have a product (i) here.

In [165]:
print "Purchase History:\n", A, "\n"
print "Bookmark History:\n", B, "\n"
print "Items similar to 4, based on purchase:\n", i * AtA, "\n"
print "Items similar to 4, based on purchase and bookmarks:\n", i * AtA + i * BtA, "\n"


Purchase History:
[[1 0 1 0 0 0]
 [1 0 1 0 1 0]
 [0 1 1 1 0 0]
 [0 1 0 1 0 1]
 [0 0 0 0 0 0]] 

Bookmark History:
[[0 0 0 1 0 0]
 [0 0 0 1 0 0]
 [0 1 0 0 0 0]
 [0 1 0 0 0 0]
 [1 0 0 0 1 1]] 

Items similar to 4, based on purchase:
[[0 2 1 0 0 1]] 

Items similar to 4, based on purchase and bookmarks:
[[2 2 3 0 1 1]] 

